# Headers

In [1]:
import pandas as pd
import time
import numpy as np



# Metrics
from sklearn.metrics import accuracy_score, f1_score

In [2]:
import torch


print("CUDA available: ", torch.cuda.is_available())
print("Number of GPUs: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")

device ='cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')


CUDA available:  False
Number of GPUs:  0
GPU Name:  No GPU available
Using cpu device


In [3]:
from utils import train_datapath, test_datapath
from utils import train_val_split

In [4]:
targets_for_test_df = pd.read_csv('data/targets_for_test.csv', index_col=0)


# TabNet

## Crude


In [5]:
treated_train_df = pd.read_csv('data/treated_train.csv')

X_train, y_train, X_val, y_val = train_val_split(treated_train_df)


In [6]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
start_time = time.time()


tabnet = TabNetClassifier()

tabnet.fit(
    X_train.values, y_train.values,
    eval_set=[(X_val.values, y_val.values)],
    eval_metric=['accuracy'],
    max_epochs=100, patience=10,
    batch_size=256, virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)


# Save the trained model
tabnet.save_model("models/tabnet_pretrained_model")

end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')



d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_tranformers\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.69048 | val_0_accuracy: 0.5293  |  0:03:33s


In [ ]:

# Load the pre-trained model
loaded_model = TabNetClassifier()
loaded_model.load_model("models/tabnet_pretrained_model.zip")

# Test the loaded model
y_pred = loaded_model.predict(X_val.values)
# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(y_val, y_pred, average='macro')
print(f'Validation F1 Macro Score: {f1_macro:.5f}')


d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_tranformers\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_tranformers\Lib\site-packages\pytorch_tabnet\abstract_model.py:454: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`.

Validation Accuracy: 0.53347
Validation F1 Macro Score: 0.39120


# TST 

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


import pandas as pd
import numpy as np
import h5py

## Dataset

In [6]:
train_df = pd.read_csv(train_datapath)
num_features = len(train_df.columns) - 1
del train_df


batch_size, sequence_len, num_features = 1024, 60, num_features  # dimensions of the dataset

## Model

In [7]:
import torch
import torch.nn as nn

class TimeSeriesTransformer(nn.Module):
    def __init__(self, num_features, d_model, num_heads, num_layers, seq_len):
        super(TimeSeriesTransformer, self).__init__()
        self.embedding = nn.Linear(num_features, d_model)
        self.transformer = nn.Transformer(d_model=d_model, nhead=num_heads, num_encoder_layers=num_layers, batch_first=True)
        self.fc_out = nn.Linear(d_model * seq_len, 1)  # Binary classification output

    def forward(self, x):
        # Embed input features
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Transformer expects [seq_len, batch, feature]
        # Pass through transformer layers
        x = self.transformer(x, x)
        x = x.permute(1, 0, 2)  # Back to [batch, seq_len, feature]
        x = x.reshape(x.size(0), -1)  # Flatten for final layer
        return self.fc_out(x)


In [8]:

import torch
import torch.nn as nn
import h5py
from tqdm import tqdm


# Training parameters
epochs = 10
batch_size = 32

# Model, loss, and optimizer initialization
d_model = 64
num_heads = 4
num_features = 10  # Adjust based on your data
seq_len = 60  # Based on your time-series window size
num_layers = 2

model = TimeSeriesTransformer(num_features=num_features, d_model=d_model, num_heads=num_heads, num_layers=num_layers, seq_len=seq_len).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())


In [ ]:
def train_model(model, criterion, optimizer, epochs, filename, device=device):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        

        # Open HDF5 file for reading
        with h5py.File(filename, 'r') as h5f:

            num_batches = len([key for key in h5f.keys() if key.startswith("batch_")])
            print(f'number of batches: {num_batches}')
            for batch_index in tqdm(range(num_batches), desc=f"Epoch {epoch+1}/{epochs}"):
                # Load data and labels for the current batch
                batch_data = h5f[f'batch_{batch_index}'][:]
                X_batch = torch.tensor(batch_data[:, :, :-1], dtype=torch.float32)  # Exclude target column
                y_batch = torch.tensor(batch_data[:, 0, -1], dtype=torch.float32)   # Target as a separate vector

                # Move data to the correct device if using a GPU
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                # Zero the gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(X_batch).squeeze()  # [batch_size]
                loss = criterion(outputs, y_batch)

                # Backward pass and optimization
                loss.backward()
                optimizer.step()

                # Track loss for the epoch
                epoch_loss += loss.item()

        # Average loss for the epoch
        avg_loss = epoch_loss / batch_index
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')



: 

In [ ]:

# Model parameters
num_features = num_features # Number of input features
d_model = 64                # Transformer embedding dimension
num_heads = 4               # Number of attention heads
num_layers = 2              # Number of transformer layers
sequence_len = sequence_len # Sequence length
print(f'Model parameters defined')
# Instantiate model, criterion, and optimizer
model = TimeSeriesTransformer(num_features, d_model, num_heads, num_layers, sequence_len)
criterion = nn.BCEWithLogitsLoss()  # For binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f'Model, criterion, and optimizer initialized')
# Call the function to train the model
train_model(model, criterion, optimizer, epochs, 'data/test_images.h5', device)



Model parameters defined
Model, criterion, and optimizer initialized
number of batches: 889


Epoch 1/10:   0%|          | 0/889 [00:00<?, ?it/s]